In [ ]:
import mysql.connector as connector

In [ ]:
from mysql.connector import pooling, Error

In [ ]:
try:
     # Connection configuration
    connection_config_dict = {
        'host': 'localhost',
        'user': 'root',
        'database': 'little_lemon_db',
        'autocommit': True
    }

    # Connect to the database using mysql.connector
    connection = connector.connect(**connection_config_dict)

    # Create a connection pool
    pool = pooling.MySQLConnectionPool(pool_name="little_lemon_pool",
                                       pool_size=3,
                                       **connection_config_dict)

    print("The connection pool is created with the name:", pool.pool_name, "and the pool size is:", pool.pool_size)

   # Create a buffered cursor
    cursor = connection.cursor(buffered=True)

    if connection.is_connected():
        
        # Use the little_lemon_db database
        use_database_query = """USE little_lemon_db;"""
        cursor.execute(use_database_query)
        print("Using little lemon database")


        
        chef_notification_query = """
        SELECT Bookings.BookingSlot, 
        CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS Guest_Name,
        CONCAT('Assigned to:' Employees.Name, '[',  Employees.Role, ']')
        WHERE Employees.Role = 'Head Chef' OR Assistant Chef
        FROM Bookings
        INNER JOIN
        Employees ON Bookings.EmployeeID=Employees.EmployeeID
        ORDER BY BookingSlot ASC LIMIT 3;
        """

        cursor.execute(chef_notification_query)
        displays = cursor.fetchall()

        print(cursor.column_names)
        for display in displays:
            print(display)



except Error as e:
    # Handle generic MySQL errors
    print("MySQL Error:", e)

except Exception as ex:
    # Handle any other type of exception not caught by previous blocks
    print("Unknown Error:", ex)

finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'connection' in locals() and connection.is_connected():
        # Returning the pool back
        connection.close()
        print("MySQL connection is closed")
